In [2]:
!pip install mysql-connector-python
!pip install beautifulsoup4

In [4]:
import mysql.connector
from mysql.connector import Error

# 데이터베이스 연결 설정
try :
    connection = mysql.connector.connect(
        host='112.167.124.86',          # 데이터베이스 서버 주소
        user='root',      # 데이터베이스 사용자 이름
        password='password',  # 사용자 비밀번호
        database='Hydro_project'   # 접근할 데이터베이스 이름
    )

    if connection.is_connected():
                print('MySQL 데이터베이스에 성공적으로 연결되었습니다.')
                cursor = connection.cursor()

except Error as e:
        print(f"MySQL 데이터베이스 에러: {e}")

MySQL 데이터베이스에 성공적으로 연결되었습니다.


## 5E 환경

### 태양광 데이터 삽입

In [7]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

while True : 
    try :
        # 현재 시간 가져오기
        current_time = datetime.now()

        # 현재 분(minute) 가져오기
        current_minute = current_time.minute
        current_sec = current_time.second

        # 5분 단위로 떨어지는지 확인
        if current_minute % 5 == 0 and current_sec <= 1:
            # 웹 페이지 URL
            date = datetime.today().strftime("%Y-%m-%d")
            url = f"http://10.21.91.183/_inc/report_list.asp?sdate={date}&edate={date}&sl_id=1&equip=IV&report_type=d&type_sd=d&inv_sel=1&data_form=5&inv_name=INV-1&wt_channel=4&cb_channel=0&equip_name=%EC%9D%B8%EB%B2%84%ED%84%B0&_search=false&nd=1716532572878&rows=10&page=1&sidx=dt&sord=desc"  # 실제 웹 페이지 URL로 교체해야 합니다.

            # 페이지로부터 HTML 가져오기
            response = requests.get(url)
            response.raise_for_status()  # 요청이 실패하면 예외를 발생시킵니다.

            # HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')
            json_str = str(soup)
            data = json.loads(json_str)

            a = dict(data['rows'][0])["cell"]

            try :
                query = "INSERT INTO 5E (일시, 전압, 전류, 입력, 전압_RS, 전압_ST_, 전압_TR_, 전류_R, 전류_S, 전류_T, 출력, 누적발전량, 주파수, 역률, 경사, 수평, 모듈, 외기) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                values = (a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8], a[10], a[11], a[12], a[13], a[14], a[15], a[16], a[17])

                cursor.execute(query, values)
                connection.commit()
            except :
                print("이미 존재하는 데이터입니다.")

    except :
        print("error 발생")

error 발생
error 발생


### 태양광 데이터 불러오기

In [10]:
import pandas as pd

# 데이터 조회
cursor.execute("SELECT * FROM 5E")
result = cursor.fetchall()

# 열 이름 리스트
columns = [
    '일시', '전압', '전류', '입력', '전압(RS)', '전압(ST)', '전압(TR)',
    '전류(R)', '전류(S)', '전류(T)', '출력', '누적발전량', '주파수',
    '역률', '경사', '수평', '모듈', '외기'
]

# 데이터를 DataFrame으로 변환
df = pd.DataFrame(result, columns=columns)

# '일시' 열을 인덱스로 설정
df.set_index('일시', inplace=True)

# 모든 가능한 타임스탬프를 포함하는 범위 생성
all_dates = pd.date_range(start='2020-01-01 00:00:00', end='2020-12-31 23:55:00', freq='5T')

# 새 데이터프레임 생성 (타임스탬프 범위를 사용하여)
full_df = pd.DataFrame(index=all_dates)

# 원본 데이터프레임과 병합
full_df = full_df.join(df, how='left')

df = full_df

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 105408 entries, 2020-01-01 00:00:00 to 2020-12-31 23:55:00
Freq: 5T
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전압      52303 non-null  object
 1   전류      52303 non-null  object
 2   입력      52303 non-null  object
 3   전압(RS)  52303 non-null  object
 4   전압(ST)  52303 non-null  object
 5   전압(TR)  52303 non-null  object
 6   전류(R)   52303 non-null  object
 7   전류(S)   52303 non-null  object
 8   전류(T)   52303 non-null  object
 9   출력      52303 non-null  object
 10  누적발전량   52303 non-null  object
 11  주파수     52303 non-null  object
 12  역률      52303 non-null  object
 13  경사      52303 non-null  object
 14  수평      52303 non-null  object
 15  모듈      52303 non-null  object
 16  외기      52303 non-null  object
dtypes: object(17)
memory usage: 18.5+ MB


In [12]:
# df['출력'] = df['출력'].astype(float)

# 모든 열을 float으로 변환
columns_to_convert = ['전압', '전류', '입력', '전압(RS)', '전압(ST)', '전압(TR)', 
                      '전류(R)', '전류(S)', '전류(T)', '출력', '누적발전량', '주파수', 
                      '역률', '경사', '수평', '모듈', '외기']

for column in columns_to_convert:
    # 먼저, 모든 열을 문자열로 변환합니다. 그 후 콤마를 제거합니다.
    df[column] = df[column].astype(str).str.replace(',', '')
    # 변환된 문자열을 실수형으로 변환합니다.
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [13]:
df

,전압,전류,입력,전압(RS),전압(ST),전압(TR),전류(R),전류(S),전류(T),출력,누적발전량,주파수,역률,경사,수평,모듈,외기
2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 23:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31 23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31 23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# 앞뒤 데이터의 평균 계산
forward = df.shift(-1)  # 바로 다음 행의 데이터
backward = df.shift(1)  # 바로 이전 행의 데이터
average = (forward + backward) / 2

# 평균값으로 결측치 채우기
df.fillna(average, inplace=True)

df.loc['2020-01-01 07:35']

전압          431.500
전류            2.950
입력            1.355
전압(RS)      391.320
전압(ST)      392.955
전압(TR)      391.505
전류(R)         5.300
전류(S)         5.000
전류(T)         5.400
출력            3.550
누적발전량     73052.030
주파수          59.950
역률           49.950
경사            0.000
수평            5.120
모듈           -2.500
외기           -1.000
Name: 2020-01-01 07:35:00, dtype: float64

In [15]:
df

,전압,전류,입력,전압(RS),전압(ST),전압(TR),전류(R),전류(S),전류(T),출력,누적발전량,주파수,역률,경사,수평,모듈,외기
2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 23:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31 23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31 23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
